### 추가 설치 필요 라이브러리
    pip install customized_konlpy

In [1]:
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pprint

In [2]:
df = pd.read_csv('review_crwl_RM.csv')
print("shape of dataframe : {}".format(df.shape))
df.head()

shape of dataframe : (11520, 4)


,time,review_txt,stars_txt,ratings
0,2019-05-30,핵과금: 엔씨 주 수입원(신화. 전설 ) Limit 중과금: 패키지 + 전설템 도전...,별표 5개 만점에 3개를 받았습니다.,3
1,2019-05-29,엘지전자 단말기로는 아예 겜을 하지말라는거요?거 해도해도 너무하네요. 삼성과 함께하...,별표 5개 만점에 1개를 받았습니다.,1
2,2019-06-03,김택진이를 재벌로 만들어준 게임의 모바일버전..김택진이의 재벌 욕심에 플레이어를 확...,별표 5개 만점에 1개를 받았습니다.,1
3,2019-06-01,구글포인트를 사용하여 게임내 아이템을 구매하였는데 어떤수단을 써도 인게임내 아이템 ...,별표 5개 만점에 3개를 받았습니다.,3
4,2019-05-28,일단 이건 게임이 아닙니다. 대다수의 아이템이 현금을 투입해야만 획득이 가능한 상태...,별표 5개 만점에 1개를 받았습니다.,1


In [3]:
df['review_txt'] = df['review_txt'].str.replace('전체 리뷰', '')
df['review_txt'] = df['review_txt'].str.replace(r'[-=+,#/\;?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》0-9]', '')
doc_to_mat = df['review_txt'].as_matrix()
doc_to_mat = list(doc_to_mat)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
okt = Okt()
pprint.pprint(okt.tagset)

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}


In [5]:
from ckonlpy.tag import Twitter
twitter = Twitter(use_twitter_dictionary=False)
twitter.add_dictionary(['사행성','리니지m', '핵과금', '무과금'], 'Noun') #명사 추가 사전
stopwords = ["이다", "있다", '진짜', '입니다', '게임','아니','하나', '정말',' 계속','그냥']
def tokenizer(raw, pos=["Noun"], stopword=stopwords): #품사 지정 
    return [
        word for word, tag in twitter.pos(raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [6]:
vectorizer = CountVectorizer(tokenizer=tokenizer, min_df=10, ngram_range=(1,2)) #tokenizer, 최소 출현문서 수, ngram 범위 uni~tri gram (최대출현문서수 추가 가능)
doc_to_vec = vectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(doc_to_vec.shape[0], doc_to_vec.shape[1]))

fit_transform, (sentence 11520, feature 747)


In [7]:
print(doc_to_vec.toarray())
features = vectorizer.get_feature_names()
#pprint.pprint(features)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
doc_to_vec_df = pd.DataFrame(doc_to_vec.toarray(), columns=features)
doc_to_vec_df.head()

,가끔,가능,가라,가면,가안,가요,가장,가지,각인,간다,...,환불,환불 요청,회사,획득,횟수,후회,휴대폰,흑기사,흑우,희귀
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [9]:
#doc_to_vec_df.to_csv('doc2vec_test.csv', index = False)

In [10]:
max_value = doc_to_vec.max(axis=0).toarray().ravel()
print(len(max_value))
sort_by_count = max_value.argsort()
features_name = np.array(vectorizer.get_feature_names())

747


In [11]:
tokencnt = doc_to_vec_df.sum(axis=0)
tokencnt = tokencnt.sort_values()
print("상위 빈도 token 50개 : \n {}".format(tokencnt[-50:]))

상위 빈도 token 50개 : 
 영변       130
해도       133
다운       134
운영       134
사냥       135
해결       139
업뎃       142
환불       142
정도       143
마켓       143
리니지m     148
지금       149
다이아      150
인증       150
삭제       156
자꾸       156
과금 유도    163
시작       166
에러       175
추억       176
설치       178
계정       179
케릭       186
확률       190
쓰레기      194
도박       197
시간       200
이벤트      222
최악       223
사람       225
아인       226
재미       232
오류       233
사행성      237
엔씨       241
아이템      245
결제       245
서버       247
생각       252
현질 유도    257
다시       306
계속       311
최고       348
유도       483
유저       503
리니지      526
업데이트     739
접속       768
현질       820
과금       836
dtype: int64


## TF_IDF

In [12]:
tfidfvectorizer = TfidfVectorizer(tokenizer=tokenizer, min_df=5, max_df = int(len(doc_to_mat)/2),ngram_range=(1,2))    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음)
tfidfvector = tfidfvectorizer.fit_transform(doc_to_mat)
print('fit_transform, (sentence {}, feature {})'.format(tfidfvector.shape[0], tfidfvector.shape[1]))
print(tfidfvector.toarray())
features = tfidfvectorizer.get_feature_names()
#print(features)

fit_transform, (sentence 11520, feature 1536)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
tfidf_vec_df = pd.DataFrame(tfidfvector.toarray(), columns=features)
tfidf_vec_df.head()

,가격,가까이,가끔,가능,가라,가면,가바,가방,가안,가왜,...,획득,횟수,횟수 초과,후회,휴대폰,흑기사,흑우,흥미,희귀,희망
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.198314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#tfidf_vec_df.to_csv('tfidf_test.csv',index=False)

In [15]:
max_tfidf = tfidf_vec_df.sum(axis=0).sort_values()
print("tf-idf 상위 50 token: \n{}".format(max_tfidf[-50:]))

tf-idf 상위 50 token: 
코드        43.679651
해결        44.640450
계정        44.730799
운영        45.349861
모바일       46.185287
삭제        46.270924
환불        46.392708
실행        49.107007
다이아       49.646079
지금        51.126132
다운로드      51.338066
다운        55.683800
역시        55.696166
확률        56.549869
리니지m      59.738041
설치        60.047314
자꾸        60.397663
서버        62.122340
케릭        63.625399
업뎃        63.709361
에러        64.366781
이벤트       65.392955
시작        65.658196
과금 유도     67.899612
결제        68.395880
사람        68.994387
시간        69.517366
오류        74.751481
생각        78.357676
영변        79.395614
다시        80.936705
아인        84.997284
아이템       87.682119
계속        91.162673
추억        91.902203
엔씨        95.501147
도박       100.780763
최악       103.023383
쓰레기      104.409735
현질 유도    104.623633
유저       125.277039
사행성      126.644768
재미       139.529396
유도       166.372232
리니지      169.484915
접속       226.869384
최고       244.525272
업데이트     277.898276
과금       284.474636

## Keyword related document

In [16]:
srch=[t for t in tokenizer("없으면") if t in features]
print("keyword token" , srch)
srch_dtm_tfidf = np.asarray(tfidfvector.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    tfidfvectorizer.vocabulary_.get(i) for i in srch]]
temp_df = pd.DataFrame(srch_dtm_tfidf, columns=srch)
temp_df = temp_df[(temp_df != 0).all(axis=1)] #하나의 token tf-idf값이 0이 있으면 제외
search_score_tfidf = temp_df.sum(axis=1)
print('related documents : {}'.format(len(search_score_tfidf)))
for i,j in zip(search_score_tfidf.sort_values(ascending=False).index, search_score_tfidf.sort_values(ascending=False).values):
    print('{} / tf-idf score : {}'.format(doc_to_mat[i], j))

keyword token []
related documents : 11520
정말 재미있어요버추얼 패드랑 터치만으로도 플레이가 되니 정말 좋네요 / tf-idf score : 0.0
돈없으면 하지마라 / tf-idf score : 0.0
최고에요 / tf-idf score : 0.0
업데이트왜안되는거임 / tf-idf score : 0.0
택진이형존경합니다 / tf-idf score : 0.0
현질 없이 힘든 게임 / tf-idf score : 0.0
영변까지는좀 적당히 뿌려라 / tf-idf score : 0.0
돈없으면 겜하지마라 / tf-idf score : 0.0
게임렉 너무합니다 / tf-idf score : 0.0
이게 겜이냐 도박이지 / tf-idf score : 0.0
무과금으론 못하겠다 / tf-idf score : 0.0
돈없으면 못하는 게임 / tf-idf score : 0.0
별점 개는 왜없음 / tf-idf score : 0.0
사행성 아니죠 뭔지 잘몰긋내 / tf-idf score : 0.0
아직도 이거 하는 흑우가 있네ㅠ / tf-idf score : 0.0
돈많은놈이 이긴다 / tf-idf score : 0.0
언넝 오픈하라고대책없는 엔씨야 / tf-idf score : 0.0
접속이 끊겼다는데 / tf-idf score : 0.0
현질안하면게임못함 / tf-idf score : 0.0
의견을 보내주세요 / tf-idf score : 0.0
환술사는 언제쯤 나오나요 / tf-idf score : 0.0
사회악같은 쓰레기 게임 반드시 천벌받는다 / tf-idf score : 0.0
너무 도박성게임ㅠㅠ / tf-idf score : 0.0
기기인증오류 해결해주세요 어제까지 재밌게했는데요 갑자기 안되네여 / tf-idf score : 0.0
왜 구매했는데 안들어오나요 / tf-idf score : 0.0
쓰레기게임 쓰레기기업 / tf-idf score : 0.0
시작을말아야 / tf-idf score : 0.0
업데이트 안되네요 겜못하는

In [17]:
srch=[t for t in tokenizer("재미 리니지") if t in features]
print("keyword token" , srch)
srch_dtm_cnt = np.asarray(doc_to_vec.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature 가 dtm 에서 가지고 있는 index값을 리턴한다
    vectorizer.vocabulary_.get(i) for i in srch]]
temp_df = pd.DataFrame(srch_dtm_cnt, columns=srch)
temp_df = temp_df[(temp_df != 0).all(axis=1)] #하나의 token count 값이 0이 있으면 제외
search_score_cnt = temp_df.sum(axis=1)
print('related documents : {}'.format(len(search_score_cnt)))
for i,j in zip(search_score_cnt.sort_values(ascending=False).index, search_score_cnt.sort_values(ascending=False).values):    
    print('{} / token count : {}'.format(doc_to_mat[i], j))

keyword token ['재미', '리니지']
related documents : 19
리니지원작느낌 그대로입니다 허나지금은년입니다 추억팔이로 요구하는 대가가 너무큽니다 현금천만원정도는투자하실 여력이있으시다면 추천합니다lt초기댓글 gt초기오픈때부터 할만큼 해봤고 이제 접으려하는데 확실한건 현금투자대비 재미가 너무 없다는거구요 저렴하게 재밌는 컨텐츠가 널렸는데 구지 이걸계속해야하나 하는 생각 갈수록 더해갑니다 혈맹지인들 때문에 접지못하다 아인값나가는거 보다가 결국 접습니다 별다른 재미도 없는데 돈은 계속투자해야하니돈은 좀쓴거같은데 재미본정도는 글쎄요pvp무한경쟁외에는 없는게임 같습니다 / token count : 4
믿고하는 리니지래서 묻지마 할랬는데 막피 이런건 상관 없는데 너무 과금 유도 하네요 무과금도 충분히 재미를 보게 해주셔야죠 무슨 과금 유저를 위한 게임이네요 잠깐 해보니 역시 리니지는 PC버전이 좋네요 / token count : 3
이제 리니지M접었다 혈질만원에 남은건 싫망과 끝없는 과금유혹 법사 랩에 접는게 아쉽지만 더이상 미련두는건 미친 짓 비싸게 뮨배우고 아이스스파이크 배워도 법산 허접 테지천지 고투사 가더 날리고 흑롭 정리 이제 리니지 안함 랩이 올라도 아인충당안되고 변신합성 수없이 해도 영웅 안주네 변신에만만원 반지에만원 해도 이상 안뜨고 수호문장도 이상안뜸 끝없는 현질에 이젠 포기 공성전 해보구 혈에서 성도 먹어 봤는데 재미는 없음 오만에살아봐도 아템먹는건 없고 리니지추억에 시작했지만 이벤트 현질유도와 아인 충당안되
 / token count : 3
오랜만에 리니지의 향수를느껴보네요 ㅎ 시간가는지모르고 하는중입니다 인터페이스토 직관적이고 편하고 리니지만의 재미도 확실하네요 ㅎ 처음해보는투사도 투사만에 매력이 있고 재미있네요 ㅎㅎㅎㅎㅎㅎ / token count : 3
리니지의 재미를 그대로 살려낸 최고의 모바일게임리니지를 해봤던 사람이라면 누구나 좋아할겁니다 / token count : 3
리니지 용의계곡이 나왓을적부터 시작한 원년멤

## Word2vec

In [37]:
from ckonlpy.tag import Postprocessor
twitter = Twitter(use_twitter_dictionary=False)
twitter.add_dictionary(['사행성','리니지m', '핵과금', '무과금'], 'Noun') #신규 단어 추가 사전, 무과금의 경우 무(명사) 과금(명사) 로 나누어지게되는데 이와 같은 단어를 위한 단어 추가
replace = {'잼': '재미'} #단어 대체 사전
postprocessor = Postprocessor(twitter, replace = replace, passtags = ['Noun']) #여러 품사 중 명사만 통과시킴

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [38]:
def sentence_morph2(text):
    tokenlist = postprocessor.pos(text) # 단어 토큰화
    temp=[]
    for word in tokenlist:
        if (len(word[0]) > 1) & (word[0] not in stopwords): #option
            temp.append((word[0])) # 해당 단어를 저장함
    return temp

In [39]:
sentence_token2 = df['review_txt'].apply(sentence_morph2)

In [40]:
from gensim.models import Word2Vec
embedding_model = Word2Vec(sentence_token2, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

In [42]:
# 특정 token(위의 case는 명사)과 가장 유사도가 큰 토큰 50개를 도출
pprint.pprint(embedding_model.most_similar(positive=["택진"], topn=50))

[('제발', 0.5697616934776306),
 ('쓰레기', 0.49158403277397156),
 ('영변', 0.4621134102344513),
 ('사람', 0.43858611583709717),
 ('확률', 0.4318772554397583),
 ('이벤트', 0.41434991359710693),
 ('엔씨', 0.3897271156311035),
 ('사행성', 0.38966286182403564),
 ('레벨', 0.3885993957519531),
 ('완전', 0.3760041296482086),
 ('인생', 0.37560927867889404),
 ('생각', 0.37477871775627136),
 ('도박', 0.37444818019866943),
 ('한번', 0.37117063999176025),
 ('유저', 0.36829909682273865),
 ('오늘', 0.3661242425441742),
 ('다운', 0.3528285026550293),
 ('개돼지', 0.3520609140396118),
 ('최고', 0.344978004693985),
 ('서버', 0.3442094027996063),
 ('아주', 0.3421030044555664),
 ('아이템', 0.3416348695755005),
 ('복구', 0.33994004130363464),
 ('인형', 0.33499887585639954),
 ('시간', 0.3343955874443054),
 ('관리', 0.3319263458251953),
 ('드랍', 0.32076936960220337),
 ('장난', 0.32023873925209045),
 ('답변', 0.3186601400375366),
 ('유도', 0.31567490100860596),
 ('이상', 0.3138505518436432),
 ('운영', 0.310166597366333),
 ('무슨', 0.2985156774520874),
 ('추억', 0.2883971631526947

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
